In [9]:
import numpy as np
import pandas as pd
import os

In [ ]:
data_path = '.'
users_df = pd.read_csv(os.path.join(data_path, 'users.csv'))
visits_df = pd.read_csv(os.path.join(data_path,'visits.csv'))
orders_df = pd.read_csv(os.path.join(data_path,'orders.csv'))
users_df = pd.read_csv(os.path.join(data_path,'users.csv'))
users_df = pd.read_csv(os.path.join(data_path,'users.csv'))